# Assignment 4
In this assignment we must read in a file of metropolitan regions and associated sports teams from [Datasets/wikipedia_data.html](Datasets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [Datasets/nfl.csv](Datasets/nfl.csv)), MLB (baseball, in [Datasets/mlb.csv](Datasets/mlb.csv)), NBA (basketball, in [Datasets/nba.csv](Datasets/nba.csv) or NHL (hockey, in [Datasets/nhl.csv](Datasets/nhl.csv)).

For each sport we have got to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html).

In [1]:
# first of all, we bring in the libraries we need
import numpy as np
import pandas as pd
import scipy.stats as stats
import re
# Filter all warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# we are going to treat whit sport team and their cities, first we could clean data related to cities
cities = pd.read_html("Datasets/wikipedia_data.html")[1]
cities.head()

,Metropolitan area,Country,Pop. rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],11,Red Bulls New York City FC,—
1,Los Angeles,United States,2,13310447,8,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,10,Galaxy Los Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [3]:
# because we are only interested in the Big 4, then
cities = cities.iloc[:-1,[0,3,5,6,7,8]]
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [4]:
# now we can see that some data has undesarible characters, so for each column we would do
for x in ['NFL', 'MLB', 'NBA', 'NHL']:
    cities[x] = cities[x].replace('\[.*\]', '', regex=True).replace('', np.nan).replace('[\—*]', np.nan, regex=True)
# Let's change the name of the column 'Population (2016 est.)[8]' by 'Population', for convenience
cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets,Yankees Mets,Knicks Nets,Rangers Islanders Devils
1,Los Angeles,13310447,Rams Chargers,Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders,Giants Athletics,Warriors,Sharks
3,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


## NHL win/loss ratio

In [5]:
# here we only need data related to NHL teams
cities_nhl = cities[['Metropolitan area', 'Population', 'NHL']].dropna()
cities_nhl = cities_nhl.rename(columns={'NHL': 'team'})
cities_nhl.head()

,Metropolitan area,Population,team
0,New York City,20153634,Rangers Islanders Devils
1,Los Angeles,13310447,Kings Ducks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [6]:
# for example, there are 3 New York's teams in one cell (the same happens to L.A.)
# so we convert that string into a only word
cities_nhl['team'].iloc[0] = 'RangersIslandersDevils'
cities_nhl['team'].iloc[1] = 'KingsDucks'
# and let's take only one word to identify the team
cities_nhl['team'] = cities_nhl['team'].str.split(' ').str[-1].astype(str)
cities_nhl.head()

,Metropolitan area,Population,team
0,New York City,20153634,RangersIslandersDevils
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [7]:
# Now let's bring in nhl data and clean it
nhl_df = pd.read_csv("Datasets/nhl.csv")
nhl_df['team'] = nhl_df['team'].replace('[\*]', '', regex=True) # take away useless characters
nhl_df = nhl_df[nhl_df['year']==2018] # we're interested on data of 2018
nhl_df = nhl_df[['team', 'W', 'L', 'year']].drop([0, 9, 18, 26]) # only these columns and rows are essential

# now we compute the win/loss ratio
nhl_df['win/loss'] = nhl_df['W'].astype(float)/(nhl_df['W'].astype(float) + nhl_df['L'].astype(float))
nhl_df['team'] = nhl_df['team'].str.split(' ').str[-1].astype(str) # take only one word to identify the team
nhl_df = nhl_df.set_index('team')
nhl_df.head()

,W,L,year,win/loss
team,,,,
Lightning,54,23,2018,0.701299
Bruins,50,20,2018,0.714286
Leafs,49,26,2018,0.653333
Panthers,44,30,2018,0.594595
Wings,30,39,2018,0.434783


In [8]:
# because there're metropolitan areas which has more than one team, we take their win/loss ratio's average
    # new york --> 'Rangers', 'Islanders', 'Devils'
W = int(nhl_df[nhl_df.index == 'Rangers'].values[0][0]) + int(nhl_df[nhl_df.index == 'Islanders'].values[0][0]) \
    + int(nhl_df[nhl_df.index == 'Devils'].values[0][0])
L = int(nhl_df[nhl_df.index == 'Rangers'].values[0][1]) + int(nhl_df[nhl_df.index == 'Islanders'].values[0][1]) \
    + int(nhl_df[nhl_df.index == 'Devils'].values[0][1])
mean = (nhl_df[nhl_df.index == 'Rangers'].values[0][-1] + nhl_df[nhl_df.index == 'Islanders'].values[0][-1] \
        + nhl_df[nhl_df.index == 'Devils'].values[0][-1])/3
nhl_new_york = {'team': 'RangersIslandersDevils',
                'W': W, 'L': L, 'year': 2018, 
                'win/loss': W/(W+L)}
nhl_new_york

{'team': 'RangersIslandersDevils',
 'W': 113,
 'L': 105,
 'year': 2018,
 'win/loss': 0.518348623853211}

In [9]:
    # los angeles --> 'Kings', 'Ducks'
W = int(nhl_df[nhl_df.index == 'Kings'].values[0][0]) + int(nhl_df[nhl_df.index == 'Ducks'].values[0][0])
L = int(nhl_df[nhl_df.index == 'Kings'].values[0][1]) + int(nhl_df[nhl_df.index == 'Ducks'].values[0][1])
mean = (nhl_df[nhl_df.index == 'Kings'].values[0][-1] + nhl_df[nhl_df.index == 'Ducks'].values[0][-1])/2
nhl_los_angeles = {'team': 'KingsDucks',
                   'W': W, 'L': L, 'year': 2018, 
                   'win/loss': W/(W+L)}
nhl_los_angeles

{'team': 'KingsDucks',
 'W': 89,
 'L': 54,
 'year': 2018,
 'win/loss': 0.6223776223776224}

In [10]:
# thus we first drop rows of the teams previously merged
nhl_df = nhl_df.drop(['Rangers', 'Islanders', 'Devils', 'Kings', 'Ducks'])

In [11]:
# and let's add the rows related to new york and los angeles
nhl_df = nhl_df.reset_index()
nhl_df = nhl_df.append([nhl_new_york, nhl_los_angeles], ignore_index=True)
nhl_df.T

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
team,Lightning,Bruins,Leafs,Panthers,Wings,Canadiens,Senators,Sabres,Capitals,Penguins,...,Stars,Blackhawks,Knights,Sharks,Flames,Oilers,Canucks,Coyotes,RangersIslandersDevils,KingsDucks
W,54,50,49,44,30,29,28,25,49,47,...,42,33,51,45,37,36,31,29,113,89
L,23,20,26,30,39,40,43,45,26,29,...,32,39,24,27,35,40,40,41,105,54
year,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018,...,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
win/loss,0.701299,0.714286,0.653333,0.594595,0.434783,0.42029,0.394366,0.357143,0.653333,0.618421,...,0.567568,0.458333,0.68,0.625,0.513889,0.473684,0.43662,0.414286,0.518349,0.622378


In [12]:
# finally, let's merge both nhl_df and cities_nhl data
merge_nhl = pd.merge(nhl_df, cities_nhl, how='outer', on='team')
merge_nhl['Population'] = merge_nhl['Population'].astype(float)
merge_nhl = merge_nhl.groupby('Metropolitan area').agg({'Population': np.mean, 'win/loss': np.mean})
merge_nhl.head()

,Population,win/loss
Metropolitan area,,
Boston,4794447.0,0.714286
Buffalo,1132804.0,0.357143
Calgary,1392609.0,0.513889
Chicago,9512999.0,0.458333
Columbus,2041520.0,0.600000


In [13]:
# and we perform the correlation between population and win/loss ratio
population_by_region = merge_nhl['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge_nhl['win/loss'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
corr1, pval1 = stats.pearsonr(population_by_region, win_loss_by_region)
print('corr =', corr1, '; pval =', pval1)

corr = 0.012308996455744264 ; pval = 0.9504308637909502


In [14]:
assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

## NBA win/loss ratio 

In [15]:
# here we only need data related to NHL teams
cities_nba = cities[['Metropolitan area', 'Population', 'NBA']].dropna()
cities_nba = cities_nba.rename(columns={'NBA': 'team'})
cities_nba.head()

,Metropolitan area,Population,team
0,New York City,20153634,Knicks Nets
1,Los Angeles,13310447,Lakers Clippers
2,San Francisco Bay Area,6657982,Warriors
3,Chicago,9512999,Bulls
4,Dallas–Fort Worth,7233323,Mavericks


In [16]:
# again there're metropolitan areas with more than one team
cities_nba['team'].iloc[0] = 'KnicksNets'
cities_nba['team'].iloc[1] = 'LakersClippers'
cities_nba['team'] = cities_nba['team'].str.split(' ').str[-1].astype(str)
cities_nba.head()

,Metropolitan area,Population,team
0,New York City,20153634,KnicksNets
1,Los Angeles,13310447,LakersClippers
2,San Francisco Bay Area,6657982,Warriors
3,Chicago,9512999,Bulls
4,Dallas–Fort Worth,7233323,Mavericks


In [17]:
# Cleaning NBA Data
nba_df = pd.read_csv("Datasets/nba.csv")
nba_df['team'] = nba_df['team'].replace(['[\*]','\(.*\)','\s*$'], '', regex=True) # take away useless characters
nba_df = nba_df[nba_df['year']==2018] # we're interested on data of 2018
nba_df = nba_df[['team', 'W', 'L', 'year']] # only these columns are essential

# now we compute the win/loss ratio
nba_df['win/loss'] = nba_df['W'].astype(float)/(nba_df['W'].astype(float) + nba_df['L'].astype(float))
nba_df['team'] = nba_df['team'].str.split(' ').str[-1].astype(str) 
nba_df = nba_df.set_index('team')
nba_df.head()

,W,L,year,win/loss
team,,,,
Raptors,59,23,2018,0.719512
Celtics,55,27,2018,0.670732
76ers,52,30,2018,0.634146
Cavaliers,50,32,2018,0.609756
Pacers,48,34,2018,0.585366


In [18]:
# now let's take the average on the team which belong of the seam metropolitan area
    # new york --> 'Knicks', 'Nets'
W = int(nba_df[nba_df.index == 'Knicks'].values[0][0]) + int(nba_df[nba_df.index == 'Nets'].values[0][0])
L = int(nba_df[nba_df.index == 'Knicks'].values[0][1]) + int(nba_df[nba_df.index == 'Nets'].values[0][1])
mean = (nba_df[nba_df.index == 'Knicks'].values[0][-1] + nba_df[nba_df.index == 'Nets'].values[0][-1])/2
nba_new_york = {'team': 'KnicksNets',
            'W': W, 'L': L, 'year': 2018, 
            'win/loss': W/(W+L)}
print(nba_new_york)

{'team': 'KnicksNets', 'W': 57, 'L': 107, 'year': 2018, 'win/loss': 0.3475609756097561}


In [19]:
    # los angeles --> 'Lakers', 'Clippers'
W = int(nba_df[nba_df.index == 'Lakers'].values[0][0]) + int(nba_df[nba_df.index == 'Clippers'].values[0][0])
L = int(nba_df[nba_df.index == 'Lakers'].values[0][1]) + int(nba_df[nba_df.index == 'Clippers'].values[0][1])
mean = (nba_df[nba_df.index == 'Lakers'].values[0][-1] + nba_df[nba_df.index == 'Clippers'].values[0][-1])/2
nba_los_angeles = {'team': 'LakersClippers',
               'W': W, 'L': L, 'year': 2018, 
               'win/loss': W/(W+L)}
print(nba_los_angeles)

{'team': 'LakersClippers', 'W': 77, 'L': 87, 'year': 2018, 'win/loss': 0.4695121951219512}


In [20]:
# let's drop these columns
nba_df = nba_df.drop(['Knicks','Nets','Lakers','Clippers'])

In [21]:
# and let's add the rows related to new york and los angeles
nba_df = nba_df.reset_index()
nba_df = nba_df.append([nba_new_york, nba_los_angeles], ignore_index=True)
nba_df.T

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
team,Raptors,Celtics,76ers,Cavaliers,Pacers,Heat,Bucks,Wizards,Pistons,Hornets,...,Pelicans,Spurs,Timberwolves,Nuggets,Kings,Mavericks,Grizzlies,Suns,KnicksNets,LakersClippers
W,59,55,52,50,48,44,44,43,39,36,...,48,47,47,46,27,24,22,21,57,77
L,23,27,30,32,34,38,38,39,43,46,...,34,35,35,36,55,58,60,61,107,87
year,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018,...,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
win/loss,0.719512,0.670732,0.634146,0.609756,0.585366,0.536585,0.536585,0.52439,0.47561,0.439024,...,0.585366,0.573171,0.573171,0.560976,0.329268,0.292683,0.268293,0.256098,0.347561,0.469512


In [22]:
# finally, let's merge both nba_df and cities data
merge_nba = pd.merge(nba_df, cities_nba, how='outer', on='team')
merge_nba['Population'] = merge_nba['Population'].astype(float)
merge_nba = merge_nba.groupby('Metropolitan area').agg({'Population': np.mean, 'win/loss': np.mean})
merge_nba.head()

,Population,win/loss
Metropolitan area,,
Atlanta,5789700.0,0.292683
Boston,4794447.0,0.670732
Charlotte,2474314.0,0.439024
Chicago,9512999.0,0.329268
Cleveland,2055612.0,0.609756


In [23]:
population_by_region = merge_nba['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge_nba['win/loss'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan a
corr2, pval2 = stats.pearsonr(population_by_region, win_loss_by_region)
print('corr =', corr2, '; pval =', pval2)

corr = -0.17657160252844614 ; pval = 0.36874741604463


In [24]:
assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

## MLB win/loss ratio

In [27]:
# here we only need data related to MLB teams
cities_mlb = cities[['Metropolitan area', 'Population', 'MLB']].dropna()
cities_mlb = cities_mlb.rename(columns={'MLB': 'team'})
# In the team column we see there are multiple teams in one rows, so lets collapse them into a single string
cities_mlb['team'].iloc[0] = 'YankeesMets'
cities_mlb['team'].iloc[1] = 'DodgersAngels'
cities_mlb['team'].iloc[2] = 'GiantsAthletics'
cities_mlb['team'].iloc[3] = 'CubsWhiteSox'
cities_mlb['team'] = cities_mlb['team'].str.split(' ').str[-1].astype(str) # let's take into account just the last word of the each team
cities_mlb.head()

,Metropolitan area,Population,team
0,New York City,20153634,YankeesMets
1,Los Angeles,13310447,DodgersAngels
2,San Francisco Bay Area,6657982,GiantsAthletics
3,Chicago,9512999,CubsWhiteSox
4,Dallas–Fort Worth,7233323,Rangers


In [30]:
# Cleaning NBA Data
mlb_df = pd.read_csv("Datasets/mlb.csv")
mlb_df['team'] = mlb_df['team'].replace(['[\*]','\(.*\)','\s*$'], '', regex=True) # take away useless characters
mlb_df = mlb_df[mlb_df['year']==2018] # we're interested on data of 2018
mlb_df = mlb_df[['team', 'W', 'L', 'year']] # only these columns are essential
mlb_df['team'].values[8] = 'WhiteSox' # renaming a team's cell

# now we compute the win/loss ratio
mlb_df['win/loss'] = mlb_df['W'].astype(float)/(mlb_df['W'].astype(float) + mlb_df['L'].astype(float))
mlb_df['team'] = mlb_df['team'].str.split(' ').str[-1].astype(str) 
mlb_df = mlb_df.set_index('team')
mlb_df.head()

,W,L,year,win/loss
team,,,,
Sox,108,54,2018,0.666667
Yankees,100,62,2018,0.617284
Rays,90,72,2018,0.555556
Jays,73,89,2018,0.450617
Orioles,47,115,2018,0.290123


In [31]:
# now let's take the average on the team which belong of the seam metropolitan area
    # new york --> 'Yankees', 'Mets'
W = int(mlb_df[mlb_df.index == 'Yankees'].values[0][0]) + int(mlb_df[mlb_df.index == 'Mets'].values[0][0])
L = int(mlb_df[mlb_df.index == 'Yankees'].values[0][1]) + int(mlb_df[mlb_df.index == 'Mets'].values[0][1])
mean = (mlb_df[mlb_df.index == 'Yankees'].values[0][-1] + mlb_df[mlb_df.index == 'Mets'].values[0][-1])/2
mlb_new_york = {'team': 'YankeesMets',
            'W': W, 'L': L, 'year': 2018, 
            'win/loss': W/(W+L)}
print(mlb_new_york)

{'team': 'YankeesMets', 'W': 177, 'L': 147, 'year': 2018, 'win/loss': 0.5462962962962963}


In [32]:
    # los angeles --> 'Dodgers', 'Angels'
W = int(mlb_df[mlb_df.index == 'Dodgers'].values[0][0]) + int(mlb_df[mlb_df.index == 'Angels'].values[0][0])
L = int(mlb_df[mlb_df.index == 'Dodgers'].values[0][1]) + int(mlb_df[mlb_df.index == 'Angels'].values[0][1])
mean = (mlb_df[mlb_df.index == 'Dodgers'].values[0][-1] + mlb_df[mlb_df.index == 'Angels'].values[0][-1])/2
mlb_los_angeles = {'team': 'DodgersAngels',
            'W': W, 'L': L, 'year': 2018, 
            'win/loss': W/(W+L)}
print(mlb_los_angeles)

{'team': 'DodgersAngels', 'W': 172, 'L': 153, 'year': 2018, 'win/loss': 0.5292307692307693}


In [33]:
    # san francisco --> 'Giants', 'Athletics'
W = int(mlb_df[mlb_df.index == 'Giants'].values[0][0]) + int(mlb_df[mlb_df.index == 'Athletics'].values[0][0])
L = int(mlb_df[mlb_df.index == 'Giants'].values[0][1]) + int(mlb_df[mlb_df.index == 'Athletics'].values[0][1])
mean = (mlb_df[mlb_df.index == 'Giants'].values[0][-1] + mlb_df[mlb_df.index == 'Athletics'].values[0][-1])/2
mlb_san_francisco = {'team': 'GiantsAthletics',
            'W': W, 'L': L, 'year': 2018, 
            'win/loss': W/(W+L)}
print(mlb_san_francisco)

{'team': 'GiantsAthletics', 'W': 170, 'L': 154, 'year': 2018, 'win/loss': 0.5246913580246914}


In [34]:
    # chicago --> 'Cubs' 'WhiteSox'
W = int(mlb_df[mlb_df.index == 'Cubs'].values[0][0]) + int(mlb_df[mlb_df.index == 'WhiteSox'].values[0][0])
L = int(mlb_df[mlb_df.index == 'Cubs'].values[0][1]) + int(mlb_df[mlb_df.index == 'WhiteSox'].values[0][1])
mean = (mlb_df[mlb_df.index == 'Cubs'].values[0][-1] + mlb_df[mlb_df.index == 'WhiteSox'].values[0][-1])/2
mlb_chicago = {'team': 'CubsWhiteSox',
            'W': W, 'L': L, 'year': 2018, 
            'win/loss': W/(W+L)}
print(mlb_chicago)

{'team': 'CubsWhiteSox', 'W': 157, 'L': 168, 'year': 2018, 'win/loss': 0.48307692307692307}


In [35]:
# let's drop these columns
mlb_df = mlb_df.drop(['Yankees','Mets','Dodgers','Angels','Giants','Athletics','Cubs','WhiteSox'])

In [36]:
# and let's add the rows related to new york, los angeles, san_francisco and chicago
mlb_df = mlb_df.reset_index()
mlb_df = mlb_df.append([mlb_new_york, mlb_los_angeles, mlb_san_francisco, mlb_chicago], ignore_index=True)
mlb_df.T

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
team,Sox,Rays,Jays,Orioles,Indians,Twins,Tigers,Royals,Astros,Mariners,...,Cardinals,Pirates,Reds,Rockies,Diamondbacks,Padres,YankeesMets,DodgersAngels,GiantsAthletics,CubsWhiteSox
W,108,90,73,47,91,78,64,58,103,89,...,88,82,67,91,82,66,177,172,170,157
L,54,72,89,115,71,84,98,104,59,73,...,74,79,95,72,80,96,147,153,154,168
year,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018,...,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
win/loss,0.666667,0.555556,0.450617,0.290123,0.561728,0.481481,0.395062,0.358025,0.635802,0.549383,...,0.54321,0.509317,0.41358,0.558282,0.506173,0.407407,0.546296,0.529231,0.524691,0.483077


In [38]:
# finally, let's merge both mlb_df and cities data
merge_mlb = pd.merge(mlb_df, cities_mlb, how='outer', on='team')
merge_mlb['Population'] = merge_mlb['Population'].astype(float)
merge_mlb = merge_mlb.groupby('Metropolitan area').agg({'Population': np.mean, 'win/loss': np.mean})
merge_mlb.head()

,Population,win/loss
Metropolitan area,,
Atlanta,5789700.0,0.555556
Baltimore,2798886.0,0.290123
Boston,4794447.0,0.666667
Chicago,9512999.0,0.483077
Cincinnati,2165139.0,0.413580


In [40]:
population_by_region = merge_mlb['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge_mlb['win/loss'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
corr3, pval3 = stats.pearsonr(population_by_region, win_loss_by_region)
print('corr =', corr3, '; pval =', pval3)

corr = 0.1505230448710485 ; pval = 0.4629669779770907


In [41]:
assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

## NFL win/loss ratio

In [43]:
# here we only need data related to NFL teams
cities_nfl = cities[['Metropolitan area', 'Population', 'NFL']].dropna()
cities_nfl = cities_nfl.rename(columns={'NFL': 'team'})
# In the team column we see there are multiple teams in one rows, so lets collapse them into a single array
cities_nfl['team'].iloc[0] = 'GiantsJets'
cities_nfl['team'].iloc[1] = 'RamsChargers'
cities_nfl['team'].iloc[2] = '49ersRaiders'
cities_nfl['team'] = cities_nfl['team'].str.split(' ').str[-1].astype(str) # let's take into account just the last word of the each team
cities_nfl.head()

,Metropolitan area,Population,team
0,New York City,20153634,GiantsJets
1,Los Angeles,13310447,RamsChargers
2,San Francisco Bay Area,6657982,49ersRaiders
3,Chicago,9512999,Bears
4,Dallas–Fort Worth,7233323,Cowboys


In [45]:
# Cleaning NFL Data
nfl_df = pd.read_csv("Datasets/nfl.csv")
nfl_df['team'] = nfl_df['team'].replace(['[\*]','\(.*\)','\s*$','[\+]'], '', regex=True) # take away useless characters
nfl_df = nfl_df[nfl_df['year']==2018] # we're interested on data of 2018
nfl_df = nfl_df[['team', 'W', 'L', 'year']].drop([0,5,10,15,20,25,30,35]) # only these columns are essential

# now we compute the win/loss ratio
nfl_df['win/loss'] = nfl_df['W'].astype(float)/(nfl_df['W'].astype(float) + nfl_df['L'].astype(float))
nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1].astype(str) 
nfl_df = nfl_df.set_index('team')
nfl_df.head()

,W,L,year,win/loss
team,,,,
Patriots,11,5,2018,0.6875
Dolphins,7,9,2018,0.4375
Bills,6,10,2018,0.3750
Jets,4,12,2018,0.2500
Ravens,10,6,2018,0.6250


In [46]:
# now let's take the average on the team which belong of the seam metropolitan area
    # new york --> 'Giants', 'Jets'
W = int(nfl_df[nfl_df.index == 'Giants'].values[0][0]) + int(nfl_df[nfl_df.index == 'Jets'].values[0][0])
L = int(nfl_df[nfl_df.index == 'Giants'].values[0][1]) + int(nfl_df[nfl_df.index == 'Jets'].values[0][1])
mean = (nfl_df[nfl_df.index == 'Giants'].values[0][-1] + nfl_df[nfl_df.index == 'Jets'].values[0][-1])/2
nfl_new_york = {'team': 'GiantsJets',
                'W': W, 'L': L, 'year': 2018, 
                'win/loss': W/(W+L)}
print(nfl_new_york)

{'team': 'GiantsJets', 'W': 9, 'L': 23, 'year': 2018, 'win/loss': 0.28125}


In [47]:
    # los angeles --> 'Rams', 'Chargers'
W = int(nfl_df[nfl_df.index == 'Rams'].values[0][0]) + int(nfl_df[nfl_df.index == 'Chargers'].values[0][0])
L = int(nfl_df[nfl_df.index == 'Rams'].values[0][1]) + int(nfl_df[nfl_df.index == 'Chargers'].values[0][1])
mean = (nfl_df[nfl_df.index == 'Rams'].values[0][-1] + nfl_df[nfl_df.index == 'Chargers'].values[0][-1])/2
nfl_los_angeles = {'team': 'RamsChargers',
                   'W': W, 'L': L, 'year': 2018, 
                   'win/loss': W/(W+L)}
print(nfl_los_angeles)

{'team': 'RamsChargers', 'W': 25, 'L': 7, 'year': 2018, 'win/loss': 0.78125}


In [48]:
    # san francisco --> '49ers', 'Raiders'
W = int(nfl_df[nfl_df.index == '49ers'].values[0][0]) + int(nfl_df[nfl_df.index == 'Raiders'].values[0][0])
L = int(nfl_df[nfl_df.index == '49ers'].values[0][1]) + int(nfl_df[nfl_df.index == 'Raiders'].values[0][1])
mean = (nfl_df[nfl_df.index == '49ers'].values[0][-1] + nfl_df[nfl_df.index == 'Raiders'].values[0][-1])/2
nfl_san_francisco = {'team': '49ersRaiders',
                     'W': W, 'L': L, 'year': 2018, 
                     'win/loss': W/(W+L)}
print(nfl_san_francisco)

{'team': '49ersRaiders', 'W': 8, 'L': 24, 'year': 2018, 'win/loss': 0.25}


In [49]:
# let's drop these columns
nfl_df = nfl_df.drop(['Giants','Jets','Rams','Chargers','49ers','Raiders'])

In [50]:
# and let's add the rows related to new york, los angeles and san francisco
nfl_df = nfl_df.reset_index()
nfl_df = nfl_df.append([nfl_new_york, nfl_los_angeles, nfl_san_francisco], ignore_index=True)
nfl_df.T

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
team,Patriots,Dolphins,Bills,Ravens,Steelers,Browns,Bengals,Texans,Colts,Titans,...,Lions,Saints,Panthers,Falcons,Buccaneers,Seahawks,Cardinals,GiantsJets,RamsChargers,49ersRaiders
W,11,7,6,10,9,7,6,11,10,9,...,6,13,7,7,5,10,3,9,25,8
L,5,9,10,6,6,8,10,5,6,7,...,10,3,9,9,11,6,13,23,7,24
year,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018,...,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
win/loss,0.6875,0.4375,0.375,0.625,0.6,0.466667,0.375,0.6875,0.625,0.5625,...,0.375,0.8125,0.4375,0.4375,0.3125,0.625,0.1875,0.28125,0.78125,0.25


In [51]:
# finally, let's merge both mlb_df and cities data
merge_nfl = pd.merge(nfl_df, cities_nfl, how='outer', on='team')
merge_nfl['Population'] = merge_nfl['Population'].astype(float)
merge_nfl = merge_nfl.groupby('Metropolitan area').agg({'Population': np.mean, 'win/loss': np.mean})
merge_nfl.head()

,Population,win/loss
Metropolitan area,,
Atlanta,5789700.0,0.4375
Baltimore,2798886.0,0.6250
Boston,4794447.0,0.6875
Buffalo,1132804.0,0.3750
Charlotte,2474314.0,0.4375


In [52]:
population_by_region = merge_nfl['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge_nfl['win/loss'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
corr4, pval4 = stats.pearsonr(population_by_region, win_loss_by_region)
print('corr =', corr4, '; pval =', pval4)

corr = 0.004922112149349409 ; pval = 0.9797833458363694


In [53]:
assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

## Question 5
In this question we shall explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How we would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Again, average values where a sport has multiple teams in one region.

In [60]:
# First of all, we going to gather all data into a single dataframe
nfl_nhl = pd.merge(merge_nfl, merge_nhl, how='outer', on='Metropolitan area')
nfl_nhl = nfl_nhl.rename(columns={'Population_x': 'Pop_nfl', 'win/loss_x': 'ratio_nfl', 'Population_y': 'Pop_nhl', 'win/loss_y': 'ratio_nhl'})
nfl_nhl_mlb = pd.merge(nfl_nhl, merge_mlb, how='outer', on='Metropolitan area')
total = pd.merge(nfl_nhl_mlb, merge_nba, how='outer', on='Metropolitan area')
total = total.rename(columns={'Population_x': 'Pop_mlb', 'win/loss_x': 'ratio_mlb', 'Population_y': 'Pop_nba', 'win/loss_y': 'ratio_nba'})
total.head()

,Pop_nfl,ratio_nfl,Pop_nhl,ratio_nhl,Pop_mlb,ratio_mlb,Pop_nba,ratio_nba
Metropolitan area,,,,,,,,
Atlanta,5789700.0,0.4375,NaN,NaN,5789700.0,0.555556,5789700.0,0.292683
Baltimore,2798886.0,0.6250,NaN,NaN,2798886.0,0.290123,NaN,NaN
Boston,4794447.0,0.6875,4794447.0,0.714286,4794447.0,0.666667,4794447.0,0.670732
Buffalo,1132804.0,0.3750,1132804.0,0.357143,NaN,NaN,NaN,NaN
Charlotte,2474314.0,0.4375,NaN,NaN,NaN,NaN,2474314.0,0.439024


In [61]:
# thus we blend each sport in pairs, and only take the ratio data
nfl_nhl = total[['ratio_nfl', 'ratio_nhl']].dropna()
nfl_mlb = total[['ratio_nfl', 'ratio_mlb']].dropna()
nfl_nba = total[['ratio_nfl', 'ratio_nba']].dropna()
nhl_mlb = total[['ratio_nhl', 'ratio_mlb']].dropna()
nhl_nba = total[['ratio_nhl', 'ratio_nba']].dropna()
mlb_nba = total[['ratio_mlb', 'ratio_nba']].dropna()

In [62]:
# Now we can perform the ttest for each combinations
# here we take the ttest relation's pval
nfl_nhl = stats.ttest_rel(nfl_nhl['ratio_nfl'], nfl_nhl['ratio_nhl'])[1]
nfl_mlb = stats.ttest_rel(nfl_mlb['ratio_nfl'], nfl_mlb['ratio_mlb'])[1]
nfl_nba = stats.ttest_rel(nfl_nba['ratio_nfl'], nfl_nba['ratio_nba'])[1]
nhl_mlb = stats.ttest_rel(nhl_mlb['ratio_nhl'], nhl_mlb['ratio_mlb'])[1]
nhl_nba = stats.ttest_rel(nhl_nba['ratio_nhl'], nhl_nba['ratio_nba'])[1]
mlb_nba = stats.ttest_rel(mlb_nba['ratio_mlb'], mlb_nba['ratio_nba'])[1]

In [63]:
# Finally, we built the dataframe with pval values for each combination
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

# Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
# df.loc["NFL","NFL"] should return np.nan
p_values['NFL'].values[0] = p_values['NBA'].values[1] = p_values['NHL'].values[2] = p_values['MLB'].values[3] = 1.0
p_values['NFL'].values[1] = p_values['NBA'].values[0] = nfl_nba
p_values['NFL'].values[2] = p_values['NHL'].values[0] = nfl_nhl
p_values['NFL'].values[3] = p_values['MLB'].values[0] = nfl_mlb

p_values['NBA'].values[2] = p_values['NHL'].values[1] = nhl_nba
p_values['NBA'].values[3] = p_values['MLB'].values[1] = mlb_nba

p_values['NHL'].values[3] = p_values['MLB'].values[2] = nhl_mlb

p_values

,NFL,NBA,NHL,MLB
NFL,1.000000,0.941792,0.030959,0.802384
NBA,0.941792,1.000000,0.022316,0.951046
NHL,0.030959,0.022316,1.000000,0.000712
MLB,0.802384,0.951046,0.000712,1.000000


In [64]:
assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"